### 关于Python
显然你应对Python有一定的熟悉，包括其基本语法，数据类型，语言特点等，如果你还不能使用Python进行程序设计，或不能避免Python中常见的一些小陷阱，或许你应该先去找个教程补充一下。这里推一个快速学习Python的教程廖雪峰的Python教程

你应该有面向对象的概念，知道类、对象、封装、多态、继承、作用域等术语的含义。

你应该对Python的科学计算包和深度学习包有一定了解，这些包包含但不限于numpy, scipy, scikit-learn, pandas...

特别地，你需要了解什么是生成器函数（generator），以及如何编写生成器函数。什么是匿名函数（lambda）

### Sequential 模型

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation

两种构造模型的方法：
- 传递一个layer的list来构造该模型
- 通过.add()的方法一个个加入

In [6]:
model1 = Sequential([
    Dense(32, input_dim=784),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])

model2 = Sequential()
model2.add(Dense(32, input_dim=784))
model2.add(Activation('relu'))

### 指定输入数据的shape

```python
model = Sequential()
model.add(Dense(32, input_shape=(784,)))

model = Sequential()
model.add(Dense(32, batch_input_shape=(None, 784)))
# note that batch dimension is "None" here,
# so the model will be able to process batches of any size.</pre>

model = Sequential()
model.add(Dense(32, input_dim=784))
```

### Merge层

多个Sequential可经由一个Merge层合并到一个输出。Merge层的输出是一个可以被添加到新 Sequential的层对象。下面这个例子将两个Sequential合并到一起：
<img src="http://keras-cn-docs.readthedocs.io/zh_CN/latest/images/two_branches_sequential_model.png" height=200 weight=200>

```python
from keras.layers import Merge

left_branch = Sequential()
left_branch.add(Dense(32, input_dim=784))

right_branch = Sequential()
right_branch.add(Dense(32, input_dim=784))

merged = Merge([left_branch, right_branch], mode='concat')

final_model = Sequential()
final_model.add(merged)
final_model.add(Dense(10, activation='softmax'))
```

其中Merge层支持一些预定义的合并模式，包括：
- sum(defualt):逐元素相加
- concat:张量串联，可以通过提供concat_axis的关键字参数指定按照哪个轴进行串联
- mul：逐元素相乘
- ave：张量平均
- dot：张量相乘，可以通过dot_axis关键字参数来指定要消去的轴
- cos：计算2D张量（即矩阵）中各个向量的余弦距离

### 训练模型
```python
final_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
final_model.fit([input_data_1, input_data_2], targets)  # we pass one data array per model input
```

### 编译

在训练模型之前，我们需要通过compile来对学习过程进行配置。compile接收三个参数：

优化器optimizer：该参数可指定为已预定义的优化器名，如rmsprop、adagrad，或一个Optimizer类的对象，详情见[optimizers](http://keras-cn-docs.readthedocs.io/zh_CN/latest/other/optimizers/)

损失函数loss：该参数为模型试图最小化的目标函数，它可为预定义的损失函数名，如categorical_crossentropy、mse，也可以为一个损失函数。详情见[objectives](http://keras-cn-docs.readthedocs.io/zh_CN/latest/other/objectives/)

指标列表metrics：对分类问题，我们一般将该列表设置为metrics=['accuracy']。指标可以是一个预定义指标的名字,也可以是一个用户定制的函数.指标函数应该返回单个张量,或一个完成metric_name - > metric_value映射的字典.请[参考性能评估](http://keras-cn-docs.readthedocs.io/zh_CN/latest/other/metrices/)

```python
# for a multi-class classification problem
model.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])

# for a binary classification problem
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['accuracy'])

# for a mean squared error regression problem
model.compile(optimizer='rmsprop',
loss='mse')
```

### 第一个模型：全连接网络

- 层对象接受张量为参数，返回一个张量。
- 输入是张量，输出也是张量的一个框架就是一个模型
- 这样的模型可以被像Keras的Sequential一样被训练

```python
from keras.layers import Input, Dense
from keras.models import Model

# this returns a tensor
inputs = Input(shape=(784,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# this creates a model that includes
# the Input layer and three Dense layers
model = Model(input=inputs, output=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels)  # starts training
```